In [1]:
library(data.table)

In [5]:
prices = fread('aws_instance_pricing/pricing1.csv')
prices[, cpu := as.numeric(`vCPU*`)]
prices[1:3]

Warning message in eval(jsub, SDenv, parent.frame()):
“NAs introduced by coercion”


Instance,vCPU*,CPU Credits/hour,Mem (GiB),Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu
<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
t3.nano,2,6,0.5,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2
t3.micro,2,12,1.0,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2
t3.small,2,24,2.0,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2


In [12]:
spot = fread('aws_instance_pricing/spot_pricing_20211129.csv')
spot[, cost2 := as.numeric(gsub('\\$','',gsub(' .*','',Cost)))]
# spot[, cost2 := gsub('\\$','',gsub(' .*','',Cost))]
spot[1:3]

Warning message in eval(jsub, SDenv, parent.frame()):
“NAs introduced by coercion”


Instance,Cost,Windows,cost2
<chr>,<chr>,<chr>,<dbl>
a1.medium,$0.0049 per Hour,N/A*,0.0049
a1.large,$0.0098 per Hour,N/A*,0.0098
a1.xlarge,$0.0197 per Hour,N/A*,0.0197


In [16]:
m = merge(spot, prices, by='Instance')
m[1:3]

Instance,Cost,Windows,cost2,vCPU*,CPU Credits/hour,Mem (GiB),Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu
<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
c5.12xlarge,$0.4854 per Hour,$2.6641 per Hour,0.4854,48,96,NA,EBS-Only,12,9500,Intel Xeon Scalable,48
c5.18xlarge,$0.7032 per Hour,$3.9961 per Hour,0.7032,72,144,NA,EBS-Only,25,19000,Intel Xeon Scalable,72
c5.24xlarge,$0.9234 per Hour,$5.3282 per Hour,0.9234,96,192,NA,EBS-Only,25,19000,Intel Xeon Scalable,96


In [18]:
m[, cost_per_cpu := cost2/cpu]
m[order(cost_per_cpu)][1:5]

Instance,Cost,Windows,cost2,vCPU*,CPU Credits/hour,Mem (GiB),Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu,cost_per_cpu
<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
t3.nano,$0.0016 per Hour,$0.0062 per Hour,0.0016,2,6,0.5,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2,0.00080
t3a.nano,$0.0016 per Hour,$0.006 per Hour,0.0016,2,6,0.5,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.00080
t3a.micro,$0.003 per Hour,$0.012 per Hour,0.0030,2,12,1.0,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.00150
t3.micro,$0.0031 per Hour,$0.0123 per Hour,0.0031,2,12,1.0,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2,0.00155
t3a.small,$0.0056 per Hour,$0.024 per Hour,0.0056,2,24,2.0,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.00280


In [19]:
setnames(m, 'Mem (GiB)', 'RAM')
m[RAM/cpu >= 2][order(cost_per_cpu)][1:5]

Instance,Cost,Windows,cost2,vCPU*,CPU Credits/hour,RAM,Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu,cost_per_cpu
<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
t3a.medium,$0.0113 per Hour,$0.0297 per Hour,0.0113,2,24,4,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.00565
t3.medium,$0.0125 per Hour,$0.0309 per Hour,0.0125,2,24,4,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2,0.00625
t2.small,$0.0069 per Hour,$0.0159 per Hour,0.0069,1,12,2,EBS-Only,Low to Moderate,,3.3 GHz Intel Scalable Processor,1,0.00690
t2.medium,$0.0139 per Hour,$0.0319 per Hour,0.0139,2,24,4,EBS-Only,Low to Moderate,,3.3 GHz Intel Scalable Processor,2,0.00695
t3a.large,$0.0226 per Hour,$0.0502 per Hour,0.0226,2,36,8,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.01130


In [20]:
m[RAM/cpu >= 4][order(cost_per_cpu)][1:5]

Instance,Cost,Windows,cost2,vCPU*,CPU Credits/hour,RAM,Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu,cost_per_cpu
<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
t3a.large,$0.0226 per Hour,$0.0502 per Hour,0.0226,2,36,8,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.011300
t3a.xlarge,$0.0458 per Hour,$0.1187 per Hour,0.0458,4,96,16,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,4,0.011450
t3a.2xlarge,$0.093 per Hour,$0.2374 per Hour,0.0930,8,192,32,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,8,0.011625
t3.2xlarge,$0.0998 per Hour,$0.247 per Hour,0.0998,8,192,32,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",8,0.012475
t3.xlarge,$0.0499 per Hour,$0.1235 per Hour,0.0499,4,96,16,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",4,0.012475
